In [1]:
from __future__ import division, print_function

In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline
# %load_ext wurlitzer


Populating the interactive namespace from numpy and matplotlib


In [3]:
import time
import numpy as np
from collections import namedtuple

import mpc_tools.mpcqp as mqp
from director import viewerclient as vc
from irispy import Polyhedron
import boxatlas.boxatlas as box
from boxatlas.contactstabilization import BoxAtlasContactStabilization

In [4]:
vis = vc.Visualizer()

In [14]:
dist_to_wall = 0.8
surfaces = [
    box.Surface(Polyhedron.fromBounds([dist_to_wall, 0], [dist_to_wall, 2]),
            Polyhedron(np.array([[1, -1], [1, 1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2 * dist_to_wall, 0], [2 * dist_to_wall, 0]), 
            Polyhedron(np.array([[-1, -1], [1, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2 * dist_to_wall, 0], [2 * dist_to_wall, 0]), 
            Polyhedron(np.array([[-1, -1], [1, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-dist_to_wall, 0], [-dist_to_wall, 2]),
            Polyhedron(np.array([[-1, -1], [-1, 1]]), np.array([0, 0]))),
]

env = box.Environment(surfaces, Polyhedron.fromBounds([-1.2, 0], [1.2, 2]))
atlas = box.BoxAtlas()
initial_state = box.BoxAtlasState(atlas)
initial_state.qcom = np.array([0, 1])
initial_state.vcom = np.array([-2, 0.])
initial_state.qlimb = map(np.array, [[0.35, 1], [0.25, 0], [-0.25, 0], [-0.35, 1]])

desired_state = initial_state.copy()
desired_state.vcom = np.zeros(2)

opt = BoxAtlasContactStabilization(initial_state, env, desired_state,
                                   num_time_steps=20)
solnData = opt.solve()

SolutionResult.kSolutionFound


In [15]:
box.planPlayback(vis, solnData)

In [18]:
# Now fix the contact sequence from the result of the previous optimization
# and run a non-hybrid optimization problem
opt = BoxAtlasContactStabilization(initial_state, env, 
                                   desired_state,
                                   num_time_steps=20,
                                   contact_assignments=solnData.contact_indicator)
# solnData = opt.solve()
# box.planPlayback(vis, solnData)

In [19]:
qp = mqp.CanonicalMPCQP.from_mathematicalprogram(opt.prog, opt.vars.all_input_variables(), opt.vars.all_state_variables())

AssertionError: Right-hand side of the equality constraints must be zero